In [8]:
import pandas as pd
import os
import preprocessing_pipeline as pppp
import librosa
import yaml
from main import *

In [9]:
# pip install pandas numpy scikit-learn librosa soundfile tqdm noisereduce pyyaml

In [10]:
dir_fsc22

'../data/fsc22'

In [11]:
df_fsc22 = pd.read_csv(os.path.join(dir_fsc22,'metadata.csv'))
df_fsc22 

,Source File Name,Dataset File Name,Class ID,Class Name
0,17548__A.wav,1_10101.wav,1,Fire
1,17548_B.wav,1_10102.wav,1,Fire
2,17548_C.wav,1_10103.wav,1,Fire
3,17548_D.wav,1_10104.wav,1,Fire
4,17548_E.wav,1_10105.wav,1,Fire
...,...,...,...,...
2020,164882__timsc__squirrel-call.wav,27_12771.wav,27,Squirrel
2021,162648__cognito-perceptu__park-bench-atmospher...,27_12772.wav,27,Squirrel
2022,122260__echobones__angry-squirrel-long.wav,27_12773.wav,27,Squirrel
2023,82828__noisecollector__angrysquirrel-creepingt...,27_12774.wav,27,Squirrel


In [12]:
df_fsc22.loc[0,'Dataset File Name']

'1_10101.wav'

In [13]:
test_file = os.path.join(dir_fsc22,'audio',df_fsc22.loc[0,'Dataset File Name'])
test_file

'../data/fsc22\\audio\\1_10101.wav'

In [14]:
signal,sr = read_signal(test_file,sr=None)

AttributeError: module 'numba' has no attribute 'core'

In [ ]:
signal.shape

(220500,)

In [ ]:
sr

44100

In [ ]:
import IPython.display as ipd

ipd.Audio(test_file)

In [ ]:
sample_rate = 44100
segment_length = 5
n_fft = 4410
n_mels = 128
n_mfcc = 20
denoise = False

params_path = "params.yml"
with open(params_path, "r") as file:
    params = yaml.safe_load(file)

algo = params["algorithm"]
batch_size = int(algo["params"]["batch_size"])
n_epochs = 50
patience = int(algo["params"]["patience"])
learning_rate = 0.01

model_dir = "./model/cnn"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
model_save_path = os.path.join(model_dir, "model.pkl")

demo_model_path = "./model/demo"
if not os.path.exists(demo_model_path):
    os.makedirs(demo_model_path)

device = get_cuda_device()

# # CNN
# PATH = "./model/basecnn200"
# model_set = torch.load(f"{PATH}/model.pt")
# clf_pre = model_set["model"]

In [ ]:
os.path.join(dir_fsc22,'audio')

'../data/fsc22\\audio'

In [ ]:
import librosa
import argparse
import pandas as pd
import numpy as np
import pickle as pkl 
import torch
import torchaudio
import torchvision
from PIL import Image

sample_rate = 44100
segment_length = 5
n_fft = 4410
n_mels = 128
n_mfcc = 20
denoise = False
data_dir = os.path.join(dir_fsc22,'audio')

def extract_spectrogram(values, clip, entries):
	for data in entries:

		num_channels = 3
		window_sizes = [25, 50, 100]
		hop_sizes = [10, 25, 50]
		centre_sec = 2.5

		specs = []
		for i in range(num_channels):
			window_length = int(round(window_sizes[i]*sample_rate/1000))
			hop_length = int(round(hop_sizes[i]*sample_rate/1000))

			clip = torch.Tensor(clip)
			spec = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate, n_fft=4410, win_length=window_length, hop_length=hop_length, n_mels=128)(clip)
			eps = 1e-6
			spec = spec.numpy()
			spec = np.log(spec+ eps)
			spec = np.asarray(torchvision.transforms.Resize((128, 250))(Image.fromarray(spec)))
			specs.append(spec)
		new_entry = {}
		new_entry["audio"] = clip.numpy()
		new_entry["values"] = np.array(specs)
		new_entry["target"] = data["target"]
		values.append(new_entry)

def extract_features(audios):
	audio_names = list(audios.filename.unique())
	values = []
	for audio in audio_names:
		clip, sr = librosa.load(os.path.join(data_dir, audio), sr=sample_rate)
		entries = audios.loc[audios["filename"]==audio].to_dict(orient="records")
		extract_spectrogram(values, clip, entries)
		print("Finished audio {}".format(audio))
	return values


In [ ]:
df_fsc22.rename(columns={'Dataset File Name': 'filename'}, inplace=True)
df_fsc22.rename(columns={'Class ID': 'target'}, inplace=True)


In [ ]:
df_fsc22

,Source File Name,filename,target,Class Name
0,17548__A.wav,1_10101.wav,1,Fire
1,17548_B.wav,1_10102.wav,1,Fire
2,17548_C.wav,1_10103.wav,1,Fire
3,17548_D.wav,1_10104.wav,1,Fire
4,17548_E.wav,1_10105.wav,1,Fire
...,...,...,...,...
2020,164882__timsc__squirrel-call.wav,27_12771.wav,27,Squirrel
2021,162648__cognito-perceptu__park-bench-atmospher...,27_12772.wav,27,Squirrel
2022,122260__echobones__angry-squirrel-long.wav,27_12773.wav,27,Squirrel
2023,82828__noisecollector__angrysquirrel-creepingt...,27_12774.wav,27,Squirrel


In [ ]:
df_fsc22['target'] = df_fsc22['target']-1

In [ ]:
num_epochs = 50

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.models import ResNet34_Weights
# Assuming the extract_features function has been run and 'values' is available

# Define a custom dataset
class AudioDataset(Dataset):
    def __init__(self, features):
        self.features = features

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        sample = self.features[idx]
        audio = sample['audio']
        image = torch.tensor(sample['values'], dtype=torch.float32)
        target = torch.tensor(sample['target'], dtype=torch.long)
        return image, target







In [ ]:
import pickle
import os

feature_file = './feature.pkl'

# Check if the feature file exists
if not os.path.exists(feature_file):
    # Assuming the extract_features function has been defined
    features = extract_features(df_fsc22)
    # Save the features to a pickle file
    with open(feature_file, 'wb') as f:
        pickle.dump(features, f)
    print(f"Features saved to {feature_file}")
else:
    # Load the features from the pickle file
    with open(feature_file, 'rb') as f:
        features = pickle.load(f)
    print(f"Features loaded from {feature_file}")

Features loaded from ./feature.pkl


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
# Perform 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 0
validation_scores = []




for train_index, valid_index in kf.split(features):
    

    ###
    # Load the pretrained ResNet model
    model = models.resnet34(weights=ResNet34_Weights.DEFAULT)   # Load the pretrained ResNet34 model
    
    # Modify the final layer to output 27 classes
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 27)
    
    # Move the model to the GPU if available
    model = model.to(device)
    ###

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    fold += 1
    train_features = [features[i] for i in train_index]
    valid_features = [features[i] for i in valid_index]

    train_dataset = AudioDataset(train_features)
    valid_dataset = AudioDataset(valid_features)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

    # Train the model
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Fold {fold}, Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Validate the model
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    validation_scores.append((accuracy, recall, precision, f1))
    print(f"Fold {fold} - Accuracy: {accuracy:.2f}, Recall: {recall:.2f}, Precision: {precision:.2f}, F1: {f1:.2f}")

# Average the validation scores
avg_scores = np.mean(validation_scores, axis=0)
print(f"Average Validation Scores - Accuracy: {avg_scores[0]:.2f}, Recall: {avg_scores[1]:.2f}, Precision: {avg_scores[2]:.2f}, F1: {avg_scores[3]:.2f}")

Fold 1, Epoch 1, Loss: 2.060528661690506
Fold 1, Epoch 2, Loss: 1.346413864808924
Fold 1, Epoch 3, Loss: 1.1377023902593875
Fold 1, Epoch 4, Loss: 0.8347862023933261
Fold 1, Epoch 5, Loss: 0.730304675359352


KeyboardInterrupt: 